In [ ]:
import pandas as pd
import numpy as np
import yfinance as yf
from scipy.stats import norm
from backtesting import Backtest, Strategy,set_bokeh_output
from datetime import datetime
set_bokeh_output(notebook=False)

# Load Options Data
df_Options = pd.read_parquet(r"...\Merged_2023_SPY_Options.parquet")
df_Options ['QUOTE_DATE'] = pd.to_datetime(df_Options['QUOTE_DATE'])
df_Options ['EXPIRE_DATE'] = pd.to_datetime(df_Options['EXPIRE_DATE'])
df_Options ['T'] = (df_Options ['EXPIRE_DATE'] - df_Options ['QUOTE_DATE']).dt.days/365
df_Options = df_Options[df_Options ['T'] > 0]

# Convert numeric columns explicitly
for col in ['C_BID', 'C_ASK', 'P_BID', 'P_ASK', 'STRIKE']:
    df_Options[col] = pd.to_numeric(df_Options[col], errors='coerce')

Nth_Closest_Expiry =12
def Get_Nth_Closest_Expiry (group, n = Nth_Closest_Expiry):
    Option_Expiry = group['EXPIRE_DATE'].unique()
    Option_Expiry_Sorted = sorted(Option_Expiry)

    if len(Option_Expiry_Sorted) >=n:
        Nth_Closest_Expiry = Option_Expiry_Sorted[n-1]
        return group[group['EXPIRE_DATE']==Nth_Closest_Expiry]
    return group[group['EXPIRE_DATE']==Option_Expiry_Sorted[-1]]

#filter to 2nd closest expiry
df_Option_Filtered = df_Options.groupby('QUOTE_DATE').apply(Get_Nth_Closest_Expiry).reset_index(drop=True)

def Historical_Volatility (Price, Window = 30):
    Log_Returns = np.log(Price/(Price.shift(1)).dropna())
    Rolling_Log_Returns = Log_Returns.rolling(window=Window).std() * np.sqrt(252)
    return Rolling_Log_Returns

# Load SPY Data
df_SPY = yf.download("SPY", start= "2022-10-15", end="2024-01-01" , progress=False)[['Close']]
df_SPY.columns = ['SPY_CLOSE']
df_SPY=df_SPY.reset_index()
df_SPY['HISTORICAL_VOLATILITY'] = Historical_Volatility(df_SPY['SPY_CLOSE'])


# Merge Data
df_Merged = pd.merge(df_Option_Filtered,df_SPY[['Date','SPY_CLOSE','HISTORICAL_VOLATILITY']],left_on='QUOTE_DATE',right_on='Date', how='left')
df_Merged = df_Merged[['QUOTE_DATE', 'EXPIRE_DATE', 'SPY_CLOSE','HISTORICAL_VOLATILITY','T','C_BID', 'C_ASK','P_BID', 'P_ASK','STRIKE','C_LAST','P_LAST']]
df_Merged['STRIKE_DIFF'] = abs(df_Merged['SPY_CLOSE'] - df_Merged["STRIKE"])



df_Merged_Strike = df_Merged.iloc[df_Merged.groupby('QUOTE_DATE')['STRIKE_DIFF'].idxmin()].drop(columns=['STRIKE_DIFF']).reset_index(drop=True)

Risk_Free_Rate = 0.05
def Black_Scholes(S, K, T, r, Sigma, Option_Type = 'Call'):
    d1 = (np.log(S / K) + (r + 0.5 * Sigma**2) * T) / (Sigma * np.sqrt(T))
    d2 = d1 - Sigma * np.sqrt(T)
    if Option_Type == 'Call':
        BS_Price = S * norm.cdf(d1) - K * np.exp(-r * T) * norm.cdf(d2)
    else:  # put
        BS_Price = K * np.exp(-r * T) * norm.cdf(-d2) - S * norm.cdf(-d1)
    return BS_Price


df_Merged_Strike['BS_CALL_HV'] = df_Merged_Strike.apply(
    lambda row: Black_Scholes(row['SPY_CLOSE'], row['STRIKE'], row['T'], Risk_Free_Rate, row['HISTORICAL_VOLATILITY'], 'Call'),
    axis=1
)

df_Merged_Strike['BS_PUT_HV'] = df_Merged_Strike.apply(
    lambda row: Black_Scholes(row['SPY_CLOSE'], row['STRIKE'], row['T'], Risk_Free_Rate, row['HISTORICAL_VOLATILITY'], 'put'),
    axis=1
)

df_Merged_Strike['CALL_DIFF'] =  (((df_Merged_Strike['C_BID'] + df_Merged_Strike['C_ASK'])/2)- df_Merged_Strike['BS_CALL_HV'])/((df_Merged_Strike['C_BID'] + df_Merged_Strike['C_ASK'])/2)
df_Merged_Strike['PUT_DIFF'] =  (((df_Merged_Strike['P_BID'] + df_Merged_Strike['P_ASK'])/2)- df_Merged_Strike['BS_PUT_HV'])/((df_Merged_Strike['P_BID'] + df_Merged_Strike['P_ASK'])/2)
df_Merged_Strike.to_csv(r"...\Merged_Strike.csv")

# Notes, 2nd closest Option prices differ largely from BS calculated Options, hence why we used n = 12

# Prepare backtesting DataFrames (keeping your original static price setup)
df_BT_Call = df_Merged_Strike.copy()
df_BT_Call['DATE'] = pd.to_datetime(df_BT_Call['QUOTE_DATE'])
df_BT_Call = df_BT_Call.set_index('DATE').sort_index()
df_BT_Call['Open'] = (df_BT_Call['C_BID'] + df_BT_Call['C_ASK']) / 2
df_BT_Call['High'] = df_BT_Call['Open']
df_BT_Call['Low'] = df_BT_Call['Open']
df_BT_Call['Close'] = df_BT_Call['Open']
df_BT_Call['Volume'] = 0

df_BT_Put = df_Merged_Strike.copy()
df_BT_Put['DATE'] = pd.to_datetime(df_BT_Put['QUOTE_DATE'])
df_BT_Put = df_BT_Put.set_index('DATE').sort_index()
df_BT_Put['Open'] = (df_BT_Put['P_BID'] + df_BT_Put['P_ASK']) / 2
df_BT_Put['High'] = df_BT_Put['Open']
df_BT_Put['Low'] = df_BT_Put['Open']
df_BT_Put['Close'] = df_BT_Put['Open']
df_BT_Put['Volume'] = 0

# Simplified Trading Strategy
class OptionsArbitrage(Strategy):
    Entry_Threshold = 0.2
    Profit_Threshold = 0.05 
    Stop_Loss = 0.05        
    Equity_Percent = 0.01

    def init(self):
        self.is_call = 'C_BID' in self.data.df.columns
        self.diff = self.data.CALL_DIFF if self.is_call else self.data.PUT_DIFF
        self.trade_history = []

    def next(self):
        current_diff = self.diff[-1]
        current_price = self.data.Close[-1]
        current_date = self.data.index[-1]

        # Check and close existing trade
        if self.trades:  # If there’s a trade open
            trade = self.trades[0]  # Only one trade allowed, so take the first
            profit_pct = (current_price - trade.entry_price) / trade.entry_price * (-1 if 'short' in trade.tag else 1)
            exit_reason = None
            
            if 'short' in trade.tag:
                if profit_pct >= self.Profit_Threshold:
                    exit_reason = "TP"
                    print(f"  Hit TP, Profit={profit_pct:.4f} on {current_date.strftime('%Y-%m-%d')}")
                elif profit_pct <= -self.Stop_Loss:                
                    exit_reason = "SL"
                    print(f"  Hit SL, Loss={profit_pct:.4f} on {current_date.strftime('%Y-%m-%d')}")
            else:
                if profit_pct >= self.Profit_Threshold:
                    exit_reason = "TP"
                    print(f"  Hit TP, Profit={profit_pct:.4f} on {current_date.strftime('%Y-%m-%d')}")
                elif profit_pct <= -self.Stop_Loss:
                    exit_reason = "SL"
                    print(f"  Hit SL, Loss={profit_pct:.4f} on {current_date.strftime('%Y-%m-%d')}")
            
            if exit_reason:
                self.trade_history.append({
                    'Type': 'Call' if self.is_call else 'Put',
                    'Direction': 'Short' if 'short' in trade.tag else 'Long',
                    'Entry_Price': trade.entry_price,
                    'Exit_Price': current_price,
                    'Size': trade.size,
                    'Profit_Pct': profit_pct,
                    'Entry_Date': trade.entry_time,
                    'Exit_Date': current_date,
                    'Exit_Reason': exit_reason
                })
                trade.close()

        # Open new trade only if no active trades
        if len(self.trades) == 0:
            equity = self.equity
            trade_value = equity * self.Equity_Percent
            size = max(1, int(trade_value / current_price))
            
            if current_diff > self.Entry_Threshold:
                self.sell(size=size, tag='call_short' if self.is_call else 'put_short')
                print(f"Opened {'Call' if self.is_call else 'Put'} Short Trade: {size} @ {current_price:.2f} on {current_date.strftime('%Y-%m-%d')}")
            elif current_diff < -self.Entry_Threshold:
                self.buy(size=size, tag='call_long' if self.is_call else 'put_long')
                print(f"Opened {'Call' if self.is_call else 'Put'} Long Trade: {size} @ {current_price:.2f} on {current_date.strftime('%Y-%m-%d')}")

    def finalize(self):
        print(f"{'Call' if self.is_call else 'Put'} Trade History Length: {len(self.trade_history)}")
        

# Run backtests
bt_call = Backtest(df_BT_Call, OptionsArbitrage, cash=100000, commission=.002, 
                  exclusive_orders=True, trade_on_close=True)
bt_put = Backtest(df_BT_Put, OptionsArbitrage, cash=100000, commission=.002, 
                 exclusive_orders=True, trade_on_close=True)

stats_call = bt_call.run()
print("\nCall Options Backtest Results:")
print(stats_call)

stats_put = bt_put.run()
print("\nPut Options Backtest Results:")
print(stats_put)

bt_call.plot(filename='call_backtest_results.html')
bt_put.plot(filename='put_backtest_results.html')

C:\Users\Admin\AppData\Local\Temp\ipykernel_7940\735333136.py:31: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df_Option_Filtered = df_Options.groupby('QUOTE_DATE').apply(Get_Nth_Closest_Expiry).reset_index(drop=True)


Opened Call Short Trade: 56 @ 17.64 on 2023-01-04
  Hit SL, Loss=-0.1012 on 2023-01-09
Opened Call Short Trade: 57 @ 17.34 on 2023-01-10
  Hit TP, Profit=0.0836 on 2023-01-17
Opened Call Short Trade: 63 @ 15.63 on 2023-01-18
  Hit SL, Loss=-0.0636 on 2023-01-20
Opened Call Short Trade: 55 @ 18.09 on 2023-01-23
  Hit TP, Profit=0.0633 on 2023-01-24
Opened Call Short Trade: 59 @ 16.73 on 2023-01-25
  Hit SL, Loss=-0.0921 on 2023-01-30
Opened Call Short Trade: 59 @ 16.78 on 2023-01-31
  Hit SL, Loss=-0.0781 on 2023-02-06
Opened Call Short Trade: 55 @ 18.12 on 2023-02-07
  Hit SL, Loss=-0.0839 on 2023-02-10
Opened Call Short Trade: 49 @ 20.05 on 2023-02-13
  Hit TP, Profit=0.0688 on 2023-02-14
Opened Call Short Trade: 54 @ 18.39 on 2023-02-15
  Hit TP, Profit=0.0514 on 2023-02-16
Opened Call Short Trade: 55 @ 17.90 on 2023-02-17
  Hit TP, Profit=0.0520 on 2023-02-22
Opened Call Short Trade: 59 @ 16.86 on 2023-02-23
  Hit TP, Profit=0.0608 on 2023-03-01
Opened Call Short Trade: 65 @ 15.25 o

GridPlot(id='p10131', ...)